Please run The following cell to start your two-hour session

In [1]:
from new_session import *
start_session()

FloatProgress(value=0.0, max=7200.0)

Session started, will remain active for two hours as counted by the progress bar


<h1>Lab 2: Visualization of Electric Field Symmetries, Flux and Gaussian Surfaces</h1>

In this lab, you will be exploring the three symmetries for which Gauss's Law can be applied to calculate the electric field: spherical, cylindrical and planar symmetries. In this exploration, you will

<ol>
    <li> Visualize the electric field produced by uniformly-charged spheres, cylinders and slabs, both inside and outside of the charged source.</li>
    <li> Choose the appropriate shape, dimensions and location of Gaussian surface needed for each case.</li>
</ol>

We have extended the code you used and edited in Lab 1 so that it can plot the E field of symmetric continuous shapes. At the very core of the code, the calculate_E() function that you wrote is calculating the E field of small contributions along the shape and then they are added together, i.e. integrating the field over the volume of the shape.


In [7]:
#To Analia and Paola, still debating whether to hide all these functions in a seprarte file and import them or just keep them.

from vpython import *

def calculate_E(q,r_S,r_P):
    e0 = 8.854187817e-12  
    k = 1.0 / (4 * pi * e0)
    rSP = sqrt(((r_P[0]-r_S[0])**2) + ((r_P[1]-r_S[1])**2) + ((r_P[2]-r_S[2])**2))
    E_x = k*q *( (r_P[0]-r_S[0]) / rSP**3) 
    E_y = k*q *( (r_P[1]-r_S[1]) / rSP**3) 
    E_z = k*q *( (r_P[2]-r_S[2]) / rSP**3)      
    return([E_x,E_y,E_z])


def draw_configuration(charges,loc,points,draw_contributions = True, mode = None):
    
    if mode !=None:
        draw_contributions = False
    for point in points:
        Es = [calculate_E(charges[index],loc[index],point) for index in range(len(charges))]
        E_tot = [sum(E) for E in zip(*Es)]
        colors = [color.blue if charge<0 else color.red for charge in charges]
        radii = [0.5e9*abs(charge) for charge in charges]
        if mode==None:
            for index in range(len(charges)):
                sphere(pos=vector(loc[index][0],loc[index][1],loc[index][2]), radius=radii[index], color=colors[index])
        Emagn = mag(vector(E_tot[0], E_tot[1],E_tot[2]))
        scale = 1
        if  Emagn >0:
            if mode !=None:
                scale = 1. / Emagn 
            E_tot = [e*scale for e in E_tot]
            if mode !=None and charges[0]<0:
                Earrow_tot=arrow(pos=vector(-E_tot[0]+point[0],-E_tot[1]+point[1],-E_tot[2]+point[2]), axis=vector(E_tot[0], E_tot[1],E_tot[2]), shaftwidth=0.05,color=color.green)
            else:
                Earrow_tot=arrow(pos=vector(point[0],point[1],point[2]), axis=vector(E_tot[0], E_tot[1],E_tot[2]), shaftwidth=0.05,color=color.green)
        if draw_contributions:
            for index in range(len(Es)):
                E = [e*scale for e in Es[index]]
                arrow(pos=vector(point[0],point[1],point[2]), axis=vector(E[0], E[1],E[2]),  shaftwidth=0.03,color=colors[index])

def canvas_with_instructions():
    scene = canvas() 
    s = "Rotate the camera by dragging with the right mouse button,\n"
    s += "To zoom, drag with the left+right mouse buttons or use the mouse pad.\n"
    s += "Touch screen: pinch/extend to zoom, swipe or two-finger rotate."
    scene.caption = s
    return scene
                
def draw_charged_sphere(charge=1e-9,radius=1,distance=2):
    scene = canvas_with_instructions() 
    col = color.blue 
    if charge>0:
        col = color.red 
    sphere(pos=vector(0,0,0), radius=radius, color=col,opacity = 0.5)
    if distance < radius:
        charge = charge*(distance**3)/(radius**3)
        
    d = distance/(sqrt(2))
    points = [[0,0,distance],[0,distance,0],[distance,0,0]]
    points+= [[0,0,-distance],[0,-distance,0],[-distance,0,0]]
    points+= [[0,d,d],[d,d,0],[d,0,d],[0,-d,-d],[-d,-d,0],[-d,0,-d]]
    points+= [[0,-d,d],[-d,d,0],[-d,0,d],[0,d,-d],[d,-d,0],[d,0,-d]]
    
    draw_configuration([charge],[[0,0,0]],points,draw_contributions = False, mode = "sphere")

def draw_charged_cylinder(lambda0=1e-9,length=10,radius=1,distance=2):
    scene = canvas_with_instructions() 
    
    col = color.blue 
    if lambda0>0:
        col = color.red 
    cylinder(pos=vector(-length/2,0,0),axis=vector(length,0,0), radius=radius, color=col,opacity = 0.5)
    if distance <radius:
        lambda0 = lambda0*(distance**2)/(radius**2)
    dx = length/1000
    charges = []
    locs = []
    for x in arange(-length/2, length/2, dx):
        charges.append(lambda0*dx)
        locs.append([x,0,0])
    d = distance/(sqrt(2))
    points = []
    dx = length/10
    for x in arange(-length/2, length/2,dx):
        points += [[x,0,distance],[x,distance,0]]

        points+= [[x,0,-distance],[x,-distance,0]]
        points+= [[x,d,d],[x,-d,-d]]
        points+= [[x,-d,d],[x,d,-d]]
    points+= [[-length/1.98,0,0],[0.05+length/1.98,0,0],[-length/1.6,0,0],[0.05+length/1.6,0,0]]
    draw_configuration(charges,locs,points,draw_contributions = False, mode = "cylinder")

def draw_charged_slab(rho=1e-9,length=10,height=10,width=1,distance=2):
    scene = canvas_with_instructions() 
    col = color.blue 
    if rho>0:
        col = color.red 
        
    box(pos=vector(0,0,0),length=length,height=height,width=width, color=col,opacity = 0.5)
    if distance <width/2:
        rho = 2*rho*(distance)/(width)
    dx = length/50
    dy = height/50
    charges = []
    locs = []
    for x in arange(-length/2, length/2, dx):
        for y in arange(-height/2,height/2,dy):
            charges.append(rho*distance*dx*dy)
            locs.append([x,y,0])
    d = distance/(sqrt(2))
    points = []
    dx = length/8
    dy = height/8
    for x in arange(-length/2, length/2,dx):
        for y in arange(-height/2, height/2,dy):
            points += [[x,y,distance],[x,y,-distance]]
    for y in arange(-int(height/2), int(height/2)):
        points+= [[-length/1.98,y,0],[0.05+length/1.98,y,0]]
    draw_configuration(charges,locs,points,draw_contributions = False, mode = "plane")

    
def draw_Gaussian_surface(mode,radius=1,base_centers=[[0,0,0],[10,0,0]]):
    if mode =="sphere":
        sphere(pos=vector(0,0,0),radius=radius,color=color.white,opacity = 0.2)
    if mode =="cylinder":
        base = base_centers[0]
        diff = [base_centers[1][i]-base[i] for i in range(3)]
        cylinder(pos=vector(base[0],base[1],base[2]),axis=vector(diff[0],diff[1],diff[2]), radius=radius, color=color.white,opacity = 0.2)


<h2>Task 1a: Visualize the Electric Field of a Uniformly-Charged Insulating Sphere</h2>

Let's first explore how the electric field of a uniformly charged insulating sphere looks like in 3D. 

Use the cell below to run the visualization using the function <b>draw_charged_sphere( ).</b>  This function draws the charged sphere with center at the origin of a coordinate system. (This coordinate system is not shown in the simulation). It also draws the E-field vectors at a "few" points in space located at a distance <b>distance_E</b> from the center of the charged sphere. This distance is one of the three arguments that you need for the function <b>draw_charged_sphere( ).</b> The three arguments that this function takes are:

<ol>
    <li><b>source_radius:</b> radius of the charged sphere (in $m$).</li>
    <li><b>charge:</b> total charge of the sphere (in $C$).</li>
    <li><b>distance_E:</b> distance between the center of the sphere and the point at which the E field is calculated and visualized (in $m$).</li>
</ol>

Note that if the charge is positive the sphere is red, and if the charge is negative the sphere is blue.

<ul>
    <li><span style="color:blue">Experiment</span> with all arguments to generate E-field vectors produced by positive and negative spheres. You can change the argument <b>distance_E</b> to see how does the E-field looks inside and outside the sphere.</li>
        <li> Keep the charge in the $nC$ order of magnitude (~1e-9 $C$)].</li>
    <li><span style="color:blue">Comment</span> on the symmetry of the arrows you obtain.</li>
</ul>


In [11]:
# Task 1a
source_radius = 3
distance_E = 1
charge = 1e-9
draw_charged_sphere(charge=charge,radius=source_radius,distance=distance_E)


<IPython.core.display.Javascript object>

In [12]:
#  Read Task 1b before attempting this part 
#draw_Gaussian_surface("sphere",radius = FILL IN) 
#draw_Gaussian_surface("cylinder",radius = FILL IN,base_centers=FILL IN)

#   Solution:
draw_Gaussian_surface("sphere",radius = distance_E)


<h2>Task 1b: Choose the Shape, Dimensions and Location of the Gaussian Surface for a Spherical Source</h2>

It's time to draw the appropriate Gaussian surface to the above visualization.

You can use the function <b>draw_Gaussian_surface( )</b>. This function adds a Gaussian surface to the above scene, it can be either a sphere or a cylinder. This function takes the following information:
<ol>
    <li>The shape of the Gaussian surface: "sphere" or "cylinder". </li>
    <li>The dimensions of the Gaussian surface: 
    <ul>
    <li>For a sphere: radius = radius of the sphere (centered at the origin for simplicity).</li>
    <li>For a cylinder: radius = radius of the two circular bases of the cylinder and base_centers = [[$x_1, y_1, z_1$],[$x_2, y_2, z_2$]] is a list of the coordinates of the centers of the two circular bases. </li>
    </ul>
    </li>
</ol>

The resulting Gaussian surface has its center at the origin of the coordinates systemBased on your observations about the symmetries of the field in task 1a, you should choose the appropriate Gaussian surface to match the symmetry of the field. 

<b>Uncomment one of the choices in the above cell by removing the "#' symbol and fill it with the correct dimensions to add the correct Gaussian surface. You should use the variables defined above (e.g. source_radius, point_distance) to fill in the missing arguments.</b> You can also copy the correct line to the cell from task 1a and run them together.

Proceed to documenting your work as follows:
<ul>
<li>Draw the Gaussian surfaces for <span style="color:blue">all four cases:</span> +/- charged sphere, and Guassian surface inside and outside the charged sphere.</li>
<li><span style="color:blue">Take screenshots</span> of 2 of your setups that include the Gaussian surfaces using a clear camera angle.</li>
<li><span style="color:blue">Describe</span>, using yor own words, how the integral for electirc flux through the Gaussian surface $\Phi = \oint \vec{E}.\vec{dA}$ simplifies because of symmetry and the correct choice of the Gaussian surface. </li>
</ul>

<h2>Task 2a: Visualize the Electric Field of a Uniformly-Charged Insulating Cylinder</h2>

In this task, you will be repeating the same steps done in Task 1 but for a uniformly charged cylinder of finite length.  You will use the function <b>draw_charged_cylinder( )</b> that draws the charged cylinder with its axis coinciding  with the x-axis and its center coinciding with the origin of the $(x,y,z)$ coordinate system. (The coordinate system is not shown in the visualization). This function also draws the E-field produced by the cylinder at a few points in space that are at a perpendicular distance, <b>distance_E</b>, from the axis of the cylinder.

The function <b>draw_charged_cylinder( )</b> takes four arguments:

<ol>
    <li><b>lambda0:</b> charge per unit length of the cylinder (in $C/m$).</li>
    <li><b>radius:</b> radius of the circular bases (in $m$).</li>
    <li><b>length:</b> total length of the cylinder (in $m$).</li>
    <li><b>distance_E:</b> perpendicular distance between the cylinder axis and the point at which the E field is calculated and visualized (in $m$). (The E field is not calculated along the axis of the cylinder).</li>
</ol>

The cylinder is red if it is positively charged and blue if negatively charged.

<ul>
    <li><span style="color:blue">Experiment</span> with all the arguments to generate the E field produced by positive and negative cylinders. Draw the E-field vectors at points inside and outside of the cylinders [Please keep the charge density in the $nC/m$ order of magnitude (~1e-9 $C/m$)].</li>
    <li><span style="color:blue">Comment</span> on the symmetry of the arrows you obtain at a particular distance. What happens at the edges of the cylinder? What happens to the E-field vectors at the points close to the surface of the cylinder vs. the E-field vectors at points far away from it?</li>
    <li>Based on your observations, <span style="color:blue">Explain</span> the reasons why approximating the cylinder as "infinite" is necessary for applying Gauss's Law, and in what cases this approximation can still be a good one. </li>
</ul>

In [22]:
#   Task 2a  
distance_E = 4
length = 6
radius = .5
density = 1e-9
draw_charged_cylinder(lambda0=density,radius=radius,length=length,distance = distance_E)



<IPython.core.display.Javascript object>

In [ ]:
#Read Task 2b before attempting this part
#draw_Gaussian_surface("sphere",radius = FILL IN) 
#draw_Gaussian_surface("cylinder",radius = FILL IN,base_centers=FILL IN)

#Solution:
draw_Gaussian_surface("cylinder",radius = distance_E,base_centers=[[-length/4,0,0],[length/4,0,0]])


<h2>Task 2b: Choose the Shape, Dimensions and Location of the Gaussian Surface for a Cylinderical Source</h2>

Use again the function <b>draw_Gaussian_surface( ) </b> to draw the correct Gaussian surface for the charged cylinder. Fill in the needed values:
<ol>
    <li>The shape: "sphere" or "cylinder".</li>
    <li>The dimensions of the Gaussian surface: 
    <ul>
    <li>For a sphere: radius = radius of the sphere (centered at the origin for simplicity).</li>
    <li>For a cylinder: radius = radius of the two circular bases of the cylinder and base_centers = [[$x_1, y_1, z_1$],[$x_2, y_2, z_2$]] is a list of the coordinates of the centers of the two circular bases. </li>
    </ul>
    </li>
</ol>

<b>Uncomment one of the choices above by removing the "#' symbol and fill it with the correct dimensions to add the correct Gaussian surface. You should use some of the variables defined above (e.g. distance, length, radius,..) to fill in the missing arguments.</b> 
<ul>
<li>Draw the Gaussian surfaces for <span style="color:blue">all four cases:</span> cylinder positively and negatively charged, and Gaussian surface inside/outside the charged cylinder.</li><li><span style="color:blue">Take screenshots</span> of 2 of your setups that include the Gaussian surfaces using a clear camera angle.</li>
<li><span style="color:blue">Describe</span>, using yor own words, how the integral for electirc flux through the Gaussian surface $\Phi = \oint \vec{E}.\vec{dA}$ simplifies because of symmetry and the correct choice of the Gaussian surface. </li>
</ul>



<h2>Task 3a: Visualize the Electric Field of a Uniformly-Charged Insulating Slab</h2>

Finally, our third symmetry is the planar one. 

Use the function <b>draw_charged_slab( ) </b> to draw a finite slab that is parallel to the x-y plane and its center coincides with the origin of the coordinate system. (The coordinate system is not shown in the visualization.) This function also draws the E-field vectors at points at a perpendicular distance, <b>distance_E</b>, with respect to the x-y plane. 

This function takes five arguments:

<ol>
    <li><b>rho: </b> charge per unit volume of the slab (in $C/m^3$).</li>
    <li><b>length: </b>slab length along x  (in $m$).</li>
    <li><b>height: </b>slab height along y  (in $m$).</li>
    <li><b>width: </b>slab width along z [should be smaller than length $\&$ height] (in $m$).</li>
    <li><b>distance_E: </b>distance above/below the x-y plane (along z) at which the E field is calculated and visualized (in $m$).</li>
</ol>


<ul>
    <li><span style="color:blue">Experiment</span> with all arguments to generate the E field for positive and negative slabs, both inside and outside of the slabs [Please keep the charge density in the $nC/m^3$ order of magnitude (~1e-9 $C/m^3$)].</li>
    <li><span style="color:blue">Comment</span> on the symmetry of the arrows you obtain at a particular distance. What happens at the edges of the slab? What happens if the points are very close to the surface of the slab vs. far away from it?</li>
    
</ul>


In [26]:
#  Task 3a
length = 10 #x
height = 10 #y
width = 2 #full width/thickness
distance_E = 6 #above/below the center of the slab in z
density = 1e-9
draw_charged_slab(rho=density,length=length,height=height,width=width,distance=distance_E)

<IPython.core.display.Javascript object>

In [25]:
#Read Task 3b before attempting this part
#draw_Gaussian_surface("sphere",radius = FILL IN) 
#draw_Gaussian_surface("cylinder",radius = FILL IN,base_centers=FILL IN)

#Solution:
draw_Gaussian_surface("cylinder",radius = 1,base_centers=[[0,0,distance],[0,0,-distance]])


<h2>Task 3b: Choose the Shape, Dimensions and Location of the Gaussian Surface for a Charged Slab</h2>

Now, choose the correct Gaussian surface and fill in the needed values:
<ol>
    <li>The shape: "sphere" or "cylinder".</li>
    <li>The dimensions of the Gaussian surface: 
    <ul>
    <li>For a sphere: radius = radius of the sphere (centered at the origin for simplicity).</li>
    <li>For a cylinder: radius = radius of the two circular bases of the cylinder and base_centers = [[$x_1, y_1, z_1$],[$x_2, y_2, z_2$]] is a list of the coordinates of the centers of the two circular bases. </li>
    </ul>
    </li>
</ol>

<b>Uncomment one of the choices above by removing the "#' symbol and fill it with the correct dimensions to add the correct Gaussian surface. You should use some of the variables defined above (e.g. distance, length, radius,..) to fill in the missing arguments.</b> 
<ul>
<li>Draw the Gaussian surfaces for <span style="color:blue">all four cases:</span>positively and negatively charged slab, at points inside and outside the slab.</li><li><span style="color:blue">Take screenshots</span> of 2 of your setups that include the Gaussian surface using a clear camera angle.</li>
<li><span style="color:blue">Describe</span>, using your own words, how the integral for electirc flux through the Gaussian surface $\Phi = \oint \vec{E}.\vec{dA}$ simplifies because of symmetry and the correct choice of the Gaussian surface.</li>
</ul>

